In [ ]:
%load_ext autoreload
%autoreload 2


import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

import pandas as pd

from analysis_variables import *

import os

In [ ]:
start = time.time()

# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'
# At Siena on villa-straylight
topdir = '/home/bellis/'


# At Bellis' home
#topdir = '/home/bellis/babar_data/bnv_plambda'

# On Bellis' laptop
#topdir = './'

#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")


# Producing tables for the skims statistics

In [ ]:
df = bat.read_in_dataset_statistics()
df

In [ ]:
dfspinfo = bat.get_SP_cross_sections_and_labels()

dfspinfo

In [ ]:
mask = dfspinfo['SP Mode']==1235
bbbar_xsec = dfspinfo[mask]['Cross section [nb]'].values[0]

mask = dfspinfo['SP Mode']==1237
bbbar_xsec += dfspinfo[mask]['Cross section [nb]'].values[0]

print(f"The total BBbar cross section is {bbbar_xsec} nb")

In [ ]:
mask = df['Data or MC']=='Data'
df[mask]

In [ ]:
mask = (df['Data or MC']=='Data') & (df['Skim']=='LambdaVeryVeryLoose')
dftmp = df[mask][['Run', 'Luminosity (Data only) 1/pb','# of events (Data or MC)', '# of events (Data or MC) NOT SURE WHICH NUMBER TO USE']]

dftmp['# of BBbar pairs'] = dftmp['Luminosity (Data only) 1/pb']*bbbar_xsec*1000

dftmp

In [ ]:

#dftmp.style.concat(df.agg(['sum']).style)

dftmp['Run'] = dftmp['Run'].astype(int).astype(str)
dftmp.loc['Total'] = dftmp.sum(numeric_only=True)

#dftmp.iloc['Total']['Run'] = 'Total'

dftmp

In [ ]:
dftmp.at['Total','Run'] = 'Total'

In [ ]:
header = []
header.append('Run')
header.append('Luminosity (1/pb)')
header.append('\# skimmed events')
header.append('\# org. events')
header.append('\# BB pairs')

caption = "Details of the numbers of events and luminosity from the {\\tt LambdaVeryVeryLoose} skim used in this analysis."
label = 'tab:dataskims'

df.style.to_latex(position_float='centering')

output = dftmp.to_latex(index=False, header=header, float_format="%.1f", caption=caption, label=label)

# Add in centering by replacing the first EOL with "EOL + \centering + EOL"
output = output.replace('\n','\n\centering\n', 1)

# Add an hline learn the bottom above the total
output = output.replace('Total','\hline\nTotal', 1)



#styler = dftmp.style
#output = styler.to_latex(caption=caption, label=label)

print(output)

# Write it out
current_dir= os.getcwd()
print(f"Writing to {current_dir}")
directory = "tables"
path= os.path.join(current_dir,directory)
if os.path.isdir(path)== False:
    os.mkdir(path)

print(f"Writing to {directory}")

outfilename = current_dir+"/"+directory+"/table_data_skim_statistics.tex"
outfile = open(outfilename,'w+')
outfile.write(output)
outfile.close()


# Flight length

In [ ]:
#data.fields

In [ ]:
#data['spmode']

In [ ]:
#mask = data['spmode'] == '-999'
mask = data['spmode'] == '998'
#mask = data['spmode'] == '1005'

bvtx = data[mask]['B_con_Vtxx']
bvty = data[mask]['B_con_Vtxy']
bvtz = data[mask]['B_con_Vtxz']

lamvtx = data[mask]['Lambda0Vtxx']
lamvty = data[mask]['Lambda0Vtxy']
lamvtz = data[mask]['Lambda0Vtxz']

lamconfl =    data[mask]['Lambda0postFitFlight']
lamconflsig = data[mask]['Lambda0postFitFlightSignificance']

bconfl =    data[mask]['BpostFitFlight']
bconflsig = data[mask]['BpostFitFlightSignificance']


blamfldx = bvtx - lamvtx
blamfldy = bvty - lamvty

blamfltrans2 = blamfldx**2 + blamfldx**2
blamfltrans = np.sqrt(blamfltrans2)

lamfltrans2 = lamvtx**2 + lamvtx**2
lamfltrans = np.sqrt(lamfltrans2)

print(bvtx.type)
print(lamvtx.type)
print(blamfltrans.type)
print(blamfltrans.type)


lamuncmass = data[mask]['Lambda0_unc_Mass']

In [ ]:
plt.figure()
plt.hist(ak.flatten(lamuncmass),bins=100)



plt.figure(figsize=(8,8))

plt.subplot(2,2,1)
plt.hist(ak.flatten(lamconfl),bins=100, range=(0,60))
plt.xlabel('Lambda constrained flight length (cm)')

plt.subplot(2,2,2)
plt.hist(ak.flatten(lamconflsig),bins=100, range=(0,300))
plt.xlabel('Lambda constrained flight length significance')

plt.subplot(2,2,3)
plt.hist(ak.flatten(bconfl),bins=100, range=(0,.2))
plt.xlabel('B constrained flight length (cm)')

plt.subplot(2,2,4)
plt.hist(ak.flatten(bconflsig),bins=100, range=(0,4))
plt.xlabel('B constrained flight length significance')



# Transverse len
plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.hist(ak.flatten(lamconfl),bins=100, range=(0,30))

plt.subplot(1,3,2)
plt.hist(ak.flatten(blamfltrans),bins=100, range=(0,5))

plt.subplot(1,3,3)
plt.hist(ak.flatten(lamfltrans),bins=100, range=(0,5))




;

In [ ]:
print(blamfltrans.type)

print(lamuncmass.type)

In [ ]:
mask = blamfltrans>1.0
#mask = lamconfl>1.0
#mask = lamconflsig > 1.0

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.hist(ak.flatten(lamuncmass),      bins=100,range=(1.105, 1.125))
plt.hist(ak.flatten(lamuncmass[mask]),bins=100,range=(1.105, 1.125))


;

In [ ]:
#lamconfl[:,0]

In [ ]:
# Count number under peak and number of estimated background

# Mass of lambda is 1.115683

mask = blamfltrans>1.0
#mask = lamconfl>1.0
#mask = lamconflsig > 1.0

plt.figure(figsize=(12,12))


peaks = []
sigs = []
bkgs = []
pcts = []
cuts = []



for i in range(0,12):
    cut = i*0.1

    mask_fl = blamfltrans > cut
    #mask_fl = lamconfl > cut
    
    width = 0.003
    
    lo = 1.115683 - width
    hi = 1.115683 + width
    
    
    #mask = blamfltrans>1.0
    #mask = lamconfl>1.0
    #mask = lamconflsig > 1.0
    
    m = lamuncmass
    mask = (m>lo) & (m<hi) & mask_fl
    
    mask_lo_sideband = (m<=lo) & (m>(lo-2*width)) & mask_fl
    mask_hi_sideband = (m>=hi) & (m<(hi+2*width)) & mask_fl
    
    npeak = len(ak.flatten(m[mask]))
    nbkglo = len(ak.flatten(m[mask_lo_sideband]))
    nbkghi = len(ak.flatten(m[mask_hi_sideband]))
    nbkg = (nbkglo + nbkghi)/2.0
    nsig = npeak - nbkg
                 
    print(f"npeak: {npeak}   nbkg: {nbkg}     nsig: {nsig}   {nbkglo}   {nbkghi}")
    peaks.append(npeak)
    sigs.append(nsig)
    bkgs.append(nbkg)
    cuts.append(cut)
                 
    plt.subplot(3,4,i+1)
    plt.hist(ak.flatten(m),      bins=100,range=(1.105, 1.125))
    plt.hist(ak.flatten(m[mask]),bins=100,range=(1.105, 1.125))
    plt.hist(ak.flatten(m[mask_lo_sideband]),bins=100,range=(1.105, 1.125), color='yellow')
    plt.hist(ak.flatten(m[mask_hi_sideband]),bins=100,range=(1.105, 1.125), color='yellow')
    
    ;

sigs = np.array(sigs)
bkgs = np.array(bkgs)
peaks = np.array(peaks)

pcts = sigs/sigs[0]

In [ ]:

plt.figure(figsize=(12, 8))
plt.subplot(2,2,1)
plt.plot(cuts, peaks,'o')

plt.subplot(2,2,2)
plt.plot(cuts, sigs,'o')

plt.subplot(2,2,3)
plt.plot(cuts, bkgs,'o')

plt.subplot(2,2,4)
plt.plot(cuts, pcts,'o')
plt.ylim(0.7,1.1)

In [ ]:
#data.fields

In [ ]:
idx = data['Lambda0d1Idx']

x = data['pcosth'][idx]

plt.hist(ak.flatten(x), bins=100)

;